# Stationary Single-Group Pedestrian Flow - Monolithic Newton Method

## 1. Introduction

This notebook implements a stationary pedestrian flow model using a **monolithic Newton method**.

Unlike the Picard iteration (which solves Helmholtz and Continuity sequentially), Newton solves the **fully coupled nonlinear system** simultaneously using automatic differentiation for the Jacobian.

**Key differences from Picard:**
- **Coupled solving**: ρ and ψ solved together in one system
- **Quadratic convergence**: Near solution, Newton converges much faster than Picard
- **Requires good initial guess**: May diverge if starting point is far from solution
- **Smooth nonlinearities required**: Uses smooth Weidmann function (no IfPos) for AD

---

In [85]:
# ========================================
# Module Import
# ========================================

from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
from ngsolve.solvers import Newton
import numpy as np
import time

In [86]:
# ========================================
# Geometry and Mesh
# ========================================

mesh_maxh = 0.05
Hcol = 1.0  # Height [m]
Hwid = 1.0  # Width [m]

rect = Rectangle(Hwid, Hcol).Face()
rect.edges.Min(X).name = "left"
rect.edges.Max(X).name = "right"
rect.edges.Max(Y).name = "entry"
rect.edges.Min(Y).name = "exit"

geom = OCCGeometry(rect, dim=2)
mesh = Mesh(geom.GenerateMesh(maxh=mesh_maxh))

print(f"Mesh: {mesh.nv} vertices, {mesh.ne} elements")

Mesh: 514 vertices, 946 elements


In [87]:
# ========================================
# Physical Parameters
# ========================================
u0 = 1.36          # Free-flow walking speed [m/s]
rho_c = 8.0        # Critical density [ped/m²]
gamma_w = 1.913    # Weidmann shape parameter [ped/m²]

# ========================================
# Regularization Parameters
# ========================================
delta = 0.22        # Viscosity parameter [m]
epsilon = 0.05      # Diffusion coefficient [m²]
eta = 1e-4         # Gradient regularization (avoid division by zero)

# ========================================
# Boundary Conditions
# ========================================
g_inflow = 0.5     # Influx at entrance [ped/(m·s)]

# ========================================
# Numerical Parameters
# ========================================
p_order = 2        # Polynomial order
max_iter = 100     # Maximum Newton iterations
tol = 1e-6         # Convergence tolerance
dampfactor = 0.5   # Newton damping factor (0.5 = take half-steps)

print("Model Parameters:")
print(f"  u0 = {u0} m/s, rho_c = {rho_c} ped/m², gamma = {gamma_w} ped/m²")
print(f"  delta = {delta} m, epsilon = {epsilon} m²")
print(f"  p_order = {p_order}, mesh_maxh = {mesh_maxh} m")
print(f"  g_inflow = {g_inflow} ped/(m·s)")
print(f"\nNewton settings:")
print(f"  dampfactor = {dampfactor}")
print(f"  max_iter = {max_iter}")
print(f"  tolerance = {tol}")

Model Parameters:
  u0 = 1.36 m/s, rho_c = 8.0 ped/m², gamma = 1.913 ped/m²
  delta = 0.22 m, epsilon = 0.05 m²
  p_order = 2, mesh_maxh = 0.05 m
  g_inflow = 0.5 ped/(m·s)

Newton settings:
  dampfactor = 0.5
  max_iter = 100
  tolerance = 1e-06


In [88]:
# ============================================================================
# PARAMETER ANALYSIS
# ============================================================================
from src import analyze_parameters

print("Running parameter analysis...\n")

results = analyze_parameters(
    u0=u0, rho_c=rho_c, gamma_w=gamma_w,
    delta=delta, epsilon=epsilon,
    mesh=mesh, mesh_maxh=mesh_maxh, p_order=p_order,
    Hwid=Hwid, Hcol=Hcol, omega=None
)

if results['stability_continuity'] in ['EXCELLENT', 'GOOD']:
    print("\n" + "="*60)
    print("✓✓ Parameters are GOOD! Proceeding with Newton solve.")
    print("="*60)
else:
    print("\n" + "="*60)
    print("⚠⚠ WARNING: Parameters may cause issues!")
    print("="*60)

Running parameter analysis...

PARAMETER ANALYSIS AND NUMERICAL RECOMMENDATIONS

1. PHYSICAL PARAMETERS

Fundamental Diagram:
  u₀ (free-flow speed)      = 1.360 m/s
  ρc (critical density)     = 8.0 ped/m²
  γ  (Weidmann parameter)   = 1.913 ped/m²

Regularization:
  δ (Helmholtz viscosity)   = 0.220 m
  ε (continuity diffusion)  = 0.050 m²

Speed at different densities:
  f(0.5) = 1.322 m/s
  f(1.0) = 1.105 m/s
  f(2.0) = 0.696 m/s
  f(4.0) = 0.289 m/s
  f(6.0) = 0.104 m/s

2. CHARACTERISTIC LENGTH SCALES

Domain:
  Width × Height = 1.00 × 1.00 m
  L (characteristic) = 1.00 m

Diffusion length (continuity):
  lε = √(ε·L/u₀) = 0.1917 m = 19.17 cm
  Physical meaning: density spreads ~19.2 cm

Screening length (Helmholtz):
  lδ = δ·u₀ = 0.2992 m = 29.92 cm
  Physical meaning: path smoothing ~29.9 cm

Helmholtz wavelength λ = 2π·δ·f(ρ):
  λ(ρ=0.5) = 1.8279 m = 182.79 cm
  λ(ρ=1.0) = 1.5274 m = 152.74 cm
  λ(ρ=2.0) = 0.9625 m = 96.25 cm
  λ(ρ=4.0) = 0.3998 m = 39.98 cm
  λ(ρ=6.0) = 0.1440

In [89]:
# ========================================
# Weidmann Fundamental Diagram - TRULY SMOOTH VERSION
# ========================================

def weidmann_speed_smooth(rho_val):
    """
    SMOOTH Weidmann fundamental diagram for Newton's automatic differentiation.
    
    Uses smooth regularization instead of IfPos:
    - Smoothly approximates max(rho, rho_min) using sqrt
    - Exponential naturally bounds speed in [0, u₀]
    
    Parameters:
    -----------
    rho_val : CoefficientFunction or trial function
        Pedestrian density [ped/m²]
    
    Returns:
    --------
    speed : CoefficientFunction
        Walking speed [m/s], smoothly bounded
    """
    rho_min = 0.01  # Minimum density for regularization
    
    # Smooth approximation of max(rho, rho_min) using sqrt
    # This avoids IfPos and is continuously differentiable
    rho_reg = (rho_val + sqrt(rho_val**2 + rho_min**2)) / 2
    
    # Weidmann formula - exponential naturally bounds speed
    speed = u0 * (1 - exp(-gamma_w * (1/rho_reg - 1/rho_c)))
    
    return speed


print("✓ SMOOTH Weidmann function defined (for Newton AD)")
print("  - No IfPos calls")
print("  - Smooth regularization: max(ρ, ρ_min) ≈ (ρ + √(ρ² + ρ_min²))/2")
print("  - All derivatives are continuous")
print("  - Safe for automatic differentiation")

✓ SMOOTH Weidmann function defined (for Newton AD)
  - No IfPos calls
  - Smooth regularization: max(ρ, ρ_min) ≈ (ρ + √(ρ² + ρ_min²))/2
  - All derivatives are continuous
  - Safe for automatic differentiation


In [90]:
# ========================================
# Finite Element Spaces - COMPOUND SPACE
# ========================================

# Separate spaces for ρ and ψ
fes_rho = H1(mesh, order=p_order)
fes_psi = H1(mesh, order=p_order, dirichlet="exit")  # ← FIXED: exit not right!

# Compound space for coupled (ρ, ψ) system
fes = fes_rho * fes_psi

# Velocity space (for post-processing)
fes_u = H1(mesh, order=p_order, dim=2)

print("Finite Element Spaces:")
print(f"  Density (rho):        {fes_rho.ndof} DOFs")
print(f"  Potential (psi):      {fes_psi.ndof} DOFs")
print(f"  Coupled (rho, psi):   {fes.ndof} DOFs")
print(f"  Velocity (u):         {fes_u.ndof} DOFs")
print(f"\n✓ Dirichlet BC on 'exit' boundary (Min(Y), where pedestrians leave)")

Finite Element Spaces:
  Density (rho):        1973 DOFs
  Potential (psi):      1973 DOFs
  Coupled (rho, psi):   3946 DOFs
  Velocity (u):         1973 DOFs

✓ Dirichlet BC on 'exit' boundary (Min(Y), where pedestrians leave)


In [91]:
# ========================================
# Grid Functions and Initial Guess
# ========================================

# Coupled grid function
gf = GridFunction(fes, name="solution")
gf_rho, gf_psi = gf.components

# Velocity (computed after solve)
gf_u = GridFunction(fes_u, name="velocity")

# Initialize with reasonable values
# ρ: start with low density
# ψ: linear from entry (y=Hcol, ψ=0) to exit (y=0, ψ=1)
gf_rho.Set(0.5)
gf_psi.Set(1 - y/Hcol)  # ← FIXED: now goes from 0 at entry to 1 at exit
gf_psi.Set(1.0, definedon=mesh.Boundaries("exit"))  # ← FIXED: exit not right!

print("Grid Functions initialized:")
print(f"  rho: min = {min(gf_rho.vec):.4f}, max = {max(gf_rho.vec):.4f}")
print(f"  psi: min = {min(gf_psi.vec):.4f}, max = {max(gf_psi.vec):.4f}")
print(f"\nInitial guess:")
print(f"  ρ = 0.5 ped/m² (reasonable initial density)")
print(f"  ψ = 1-y/Hcol (linear from entry to exit)")
print(f"  ψ(entry at y={Hcol}) = 0")
print(f"  ψ(exit at y=0) = 1")

Grid Functions initialized:
  rho: min = -0.0000, max = 0.5000
  psi: min = -0.0000, max = 1.0000

Initial guess:
  ρ = 0.5 ped/m² (reasonable initial density)
  ψ = 1-y/Hcol (linear from entry to exit)
  ψ(entry at y=1.0) = 0
  ψ(exit at y=0) = 1


In [92]:
# ========================================
# Monolithic Newton Residual
# ========================================

print("Setting up coupled residual for Newton...\n")

# Get trial and test functions for COUPLED system
(u_rho, u_psi), (v_w, v_phi) = fes.TnT()

# Compute velocity from trial functions (for automatic differentiation)
grad_psi_trial = grad(u_psi)
grad_psi_norm_sq = InnerProduct(grad_psi_trial, grad_psi_trial)
grad_psi_norm = sqrt(grad_psi_norm_sq + eta)

# Speed using SMOOTH Weidmann (critical for AD!)
speed = weidmann_speed_smooth(u_rho)
u_trial = speed * grad_psi_trial / grad_psi_norm

# Helmholtz coefficient
f_rho_trial = weidmann_speed_smooth(u_rho)
kappa_sq = 1.0 / (delta**2 * f_rho_trial**2)

# Normal vector for boundary terms
n = specialcf.normal(2)

# Define coupled residual R(ρ, ψ)
R = BilinearForm(fes, symmetric=False)

# Helmholtz residual terms
R += grad(u_psi) * grad(v_phi) * dx
R += kappa_sq * u_psi * v_phi * dx
R += (1.0 / (u0 * delta)) * u_psi * v_phi * ds("entry")

# Continuity residual terms
R += epsilon * grad(u_rho) * grad(v_w) * dx
R += -u_rho * (u_trial * grad(v_w)) * dx
# EXIT BOUNDARY CONDITION: Compute u·n on boundary using Trace
# u_trial_boundary = speed * grad(u_psi).Trace() / ||grad(u_psi).Trace()||
grad_psi_trace = grad(u_psi).Trace()
grad_psi_trace_norm = sqrt(InnerProduct(grad_psi_trace, grad_psi_trace) + eta)
u_trial_boundary = speed * grad_psi_trace / grad_psi_trace_norm
R += u_rho * InnerProduct(u_trial_boundary, n) * v_w * ds("exit")  # ← FIXED: exit not right!
# INFLOW BOUNDARY CONDITION
R += -g_inflow * v_w * ds("entry")

print("✓ Coupled residual assembled:")
print("  - Helmholtz: ∇ψ·∇φ + κ²(ρ)ψφ + Robin BC at entry")
print("  - Continuity: ε∇ρ·∇w - ρu·∇w - inflow at entry")
print("  - EXIT BC: ∫(ρu·n)w ds('exit') at y=0 [MASS BALANCE!]")
print("  - Coupling: u = f_smooth(ρ) ∇ψ/||∇ψ||")
print("  - Using Trace() for boundary gradient evaluation")
print("  - Using TRULY SMOOTH Weidmann (no IfPos)")
print("\n→ NGSolve will compute Jacobian via automatic differentiation")

Setting up coupled residual for Newton...

✓ Coupled residual assembled:
  - Helmholtz: ∇ψ·∇φ + κ²(ρ)ψφ + Robin BC at entry
  - Continuity: ε∇ρ·∇w - ρu·∇w - inflow at entry
  - EXIT BC: ∫(ρu·n)w ds('exit') at y=0 [MASS BALANCE!]
  - Coupling: u = f_smooth(ρ) ∇ψ/||∇ψ||
  - Using Trace() for boundary gradient evaluation
  - Using TRULY SMOOTH Weidmann (no IfPos)

→ NGSolve will compute Jacobian via automatic differentiation


In [93]:
# ========================================
# Solve with Monolithic Newton
# ========================================

print("="*60)
print("MONOLITHIC NEWTON SOLVER")
print("="*60)
print(f"\nSettings:")
print(f"  dampfactor: {dampfactor}")
print(f"  maxerr: {tol}")
print(f"  maxit: {max_iter}")
print(f"  inverse: sparsecholesky")
print(f"\nStarting Newton solve...\n")

t_start = time.time()

try:
    with TaskManager():
        Newton(
            R, gf,
            freedofs=fes.FreeDofs(),
            maxerr=tol,
            maxit=max_iter,
            printing=True,
            dampfactor=0.5,
            inverse="sparsecholesky"
        )
    success = True
except Exception as e:
    print(f"\n✗ Newton failed: {e}")
    success = False

t_end = time.time()
wall_time = t_end - t_start

print("\n" + "="*60)
if success:
    print(f"✓ Newton solver completed")
else:
    print(f"✗ Newton solver failed to converge")
print(f"  Wall time: {wall_time:.4f} s")
print("="*60)

MONOLITHIC NEWTON SOLVER

Settings:
  dampfactor: 0.5
  maxerr: 1e-06
  maxit: 100
  inverse: sparsecholesky

Starting Newton solve...

Newton iteration  0


err =  5.091572680091349
Newton iteration  1
err =  2.749675663062778
Newton iteration  2
err =  12.879825605615897
Newton iteration  3
err =  121.65970040458146
Newton iteration  4
err =  921.7541068615373
Newton iteration  5
err =  6383.118152877893
Newton iteration  6
err =  11680.38928677113
Newton iteration  7
err =  88538.47207162382
Newton iteration  8
err =  1691496.7517955762
Newton iteration  9
err =  nan
Newton iteration  10
err =  nan
Newton iteration  11
err =  nan
Newton iteration  12
err =  nan
Newton iteration  13
err =  nan
Newton iteration  14
err =  nan
Newton iteration  15
err =  nan
Newton iteration  16
err =  nan
Newton iteration  17
err =  nan
Newton iteration  18
err =  nan
Newton iteration  19
err =  nan
Newton iteration  20
err =  nan
Newton iteration  21
err =  nan
Newton iteration  22
err =  nan
Newton iteration  23
err =  nan
Newton iteration  24
err =  nan
Newton iteration  25
err =  nan
Newton iteration  26
err =  nan
Newton iteration  27
err =  nan
Newto

In [94]:
# ========================================
# Compute Velocity Field
# ========================================

# Compute velocity from final solution using GridFunctions
grad_psi_cf = grad(gf_psi)
grad_psi_norm_sq_cf = InnerProduct(grad_psi_cf, grad_psi_cf)
grad_psi_norm_cf = sqrt(grad_psi_norm_sq_cf + eta)

# Use smooth Weidmann for consistency
speed_cf = weidmann_speed_smooth(gf_rho)
u_cf = speed_cf * grad_psi_cf / grad_psi_norm_cf

gf_u.Set(u_cf)

print("✓ Velocity field computed")

✓ Velocity field computed


In [95]:
# ========================================
# Solution Statistics
# ========================================

print("="*60)
print("SOLUTION STATISTICS")
print("="*60)

domain_area = Integrate(1.0, mesh)
rho_integral = Integrate(gf_rho * dx, mesh)
rho_mean = rho_integral / domain_area

# Interpolate to P1 for min/max
fes_p1 = H1(mesh, order=1)
gf_rho_p1 = GridFunction(fes_p1)
gf_rho_p1.Set(gf_rho)

print(f"\nDensity (ρ):")
print(f"  integral = {rho_integral:.6f} ped")
print(f"  mean = {rho_mean:.6f} ped/m²")
print(f"  min ≈ {min(gf_rho_p1.vec):.6f} ped/m² (at vertices)")
print(f"  max ≈ {max(gf_rho_p1.vec):.6f} ped/m² (at vertices)")

print(f"\nDomain area: {domain_area:.6f} m²")
print("="*60)

SOLUTION STATISTICS

Density (ρ):
  integral = nan ped
  mean = nan ped/m²
  min ≈ nan ped/m² (at vertices)
  max ≈ nan ped/m² (at vertices)

Domain area: 1.000000 m²


In [96]:
# ========================================
# Visualization
# ========================================

print("\nGenerating visualizations...")

Draw(gf_rho, mesh, "density")
print("  ✓ Density field (ρ)")

Draw(gf_psi, mesh, "potential")
print("  ✓ Potential field (ψ)")

Draw(gf_u, mesh, "velocity", vectors={"grid_size": 30})
print("  ✓ Velocity vector field (u)")

print("\n✓ Visualization complete!")


Generating visualizations...


WebGuiWidget(layout=Layout(height='5vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.240…

  ✓ Density field (ρ)


WebGuiWidget(layout=Layout(height='5vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.240…

  ✓ Potential field (ψ)


WebGuiWidget(layout=Layout(height='5vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.240…

  ✓ Velocity vector field (u)

✓ Visualization complete!
